# Exercice 12:

## Création d'un CNN

[lien Nowledgeable](https://nowledgeable.com/invitation/student/join-module/9c8ec467-686a-44cd-a2f2-85cf174a79ad) <br>
A l'aide de la transformée en ondelettes continue, transformez des signaux ECG en scaleograms et puis utilisez un modèle CNN pour classifier des images.


In [1]:
!pip install wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 2.2 MB/s eta 0:00:00


## Imports des librairies

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import tensorflow as tf

import wfdb
import pywt
from scipy import signal as scipy_signal


Afin de pouvoir récupérer les signaux issus des datasets de physionet, utilisons `load_signal_using_wfdb`

In [3]:
def load_signal_using_wfdb(file, start, end, channel, pn_dir):
    record = wfdb.rdrecord(file, sampfrom = start, sampto = end, channels=[channel], pn_dir=pn_dir)
    data = record.p_signal.reshape(-1)
    return data

In [4]:
# recupération des donnees
database_names = ['nsrdb','mitdb','chfdb']
n_label = len(database_names)

In [5]:
# recupération des noms des fichiers et des labels
record_names = []
labels = []

for i, name in enumerate(database_names):
    current_record_names = wfdb.get_record_list(name)
    record_names += current_record_names
    labels += [i] * len(current_record_names)

assert len(record_names) == len(labels)

In [6]:
frequencies = []
for record_name, label in zip(record_names, labels):
    header = wfdb.rdheader(record_name, database_names[label])
    frequencies.append(header.fs)

In [7]:
def resample_signal(input_signal, original_rate, target_rate):

    # Calculating the number of samples in the resampled signal
    num_samples = int((target_rate * len(input_signal)) / original_rate)

    # Resampling the input signal to match the target sampling rate
    resampled_signal = scipy_signal.resample(input_signal, num_samples)

    return resampled_signal

Passons à l'extraction de la donnée:

In [8]:
def generate_data(sample_size, scale_size, nb_samples_per_signal, labels, record_names, database_names, file_path=None, frequencies=[], common_sampling_rate=128):

    scales = range(1, scale_size + 1)
    waveletname = 'morl'
    signal_ds = []
    y_train = np.zeros(nb_samples_per_signal * len(labels))
    num_records = len(record_names)

    # Charger et traiter les signaux
    for i in range(num_records):
        record_name, label, frequency = record_names[i], labels[i], frequencies[i]
        pn_dir = database_names[label]
        signal = load_signal_using_wfdb(record_name, start=0, end=nb_samples_per_signal * frequency, channel=0, pn_dir=pn_dir)
        signal = resample_signal(signal, frequency, common_sampling_rate)
        signal_ds.extend(np.split(signal, nb_samples_per_signal))
        y_train[i * nb_samples_per_signal: (i + 1) * nb_samples_per_signal] = label

    signal_ds = np.array(signal_ds)

    # Calcul des coefficients de transformation en ondelette continue
    X_train = [pywt.cwt(signal, scales, waveletname, 1)[0] for signal in signal_ds]
    X_train = np.expand_dims(X_train, axis=-1)
    X_tensor = tf.convert_to_tensor(X_train)

    # Conversion des étiquettes en encodage one-hot et création d'un tenseur
    num_classes = len(np.unique(y_train))
    y_tensor = tf.convert_to_tensor(tf.keras.utils.to_categorical(y_train, num_classes))

    return X_tensor, y_tensor


In [9]:
sample_size = 64
scale_size = 64
nb_samples_per_signal = 64
common_sampling_rate = 64

X_tensor, y_tensor = generate_data(sample_size = sample_size,scale_size = scale_size,nb_samples_per_signal = nb_samples_per_signal, labels= labels, record_names = record_names,database_names=database_names,file_path='signals_csv',frequencies = frequencies ,common_sampling_rate = common_sampling_rate)
assert X_tensor.shape[0] == y_tensor.shape[0]

In [11]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import  Dense, Dropout, Conv2D, Input, MaxPooling2D,Flatten

def build_model(input_shape,num_class):
    model = Sequential()
    model.add(Input(shape=input_shape))

    model.add(Conv2D(filters=64,kernel_size=5, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.5))

    model.add(Conv2D(filters=32,kernel_size=3, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Dropout(0.5))

    model.add(Conv2D(filters=32,kernel_size=3, activation='relu'))
    model.add(MaxPooling2D())

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_class,activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model
model = build_model(X_tensor.shape[1:],y_tensor.shape[1])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 60, 60, 64)        1664      
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 30, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 30, 30, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 32)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 14, 14, 32)        0

In [12]:
X_tensor.shape,y_tensor.shape

(TensorShape([5184, 64, 64, 1]), TensorShape([5184, 3]))

In [13]:
model.fit(X_tensor,y_tensor,validation_split=0.2,epochs=5)

Epoch 1/5
130/130 [==============================] - 37s 272ms/step - loss: 0.4217 - accuracy: 0.8040 - val_loss: 9.8525 - val_accuracy: 0.0501
Epoch 2/5
130/130 [==============================] - 37s 283ms/step - loss: 0.2658 - accuracy: 0.8862 - val_loss: 13.9665 - val_accuracy: 0.0511
Epoch 3/5
130/130 [==============================] - 36s 276ms/step - loss: 0.2302 - accuracy: 0.9062 - val_loss: 12.8478 - val_accuracy: 0.0328
Epoch 4/5
130/130 [==============================] - 35s 267ms/step - loss: 0.2251 - accuracy: 0.9002 - val_loss: 10.1888 - val_accuracy: 0.0444
Epoch 5/5
130/130 [==============================] - 36s 274ms/step - loss: 0.2035 - accuracy: 0.9142 - val_loss: 13.7837 - val_accuracy: 0.0444


Nous observons une divergence notable entre l'accuracy de l'entraînement et celle de la validation, indiquant un overfitting significatif du modèle. Cela signifie que bien que le modèle ait appris à perfection les données d'entraînement, il échoue à généraliser son apprentissage à de nouvelles données inconnues, mises à l'épreuve durant la phase de validation.

Analyse
Cette situation pourrait découler de plusieurs raisons. Premièrement, il est possible que le modèle choisi ne soit pas le plus adapté à la nature de nos données. Un modèle trop complexe avec un grand nombre de paramètres peut facilement mémoriser les données d'entraînement mais sera inefficace pour interpréter de nouvelles données. Deuxièmement, il se pourrait également que la préparation des données ait été suboptimale, avec des choix de dimensionnalité ou d'échantillonnage qui n’alignent pas bien avec les propriétés intrinsèques des données.